# Optimization of Keras using hyperopt

In [2]:
from __future__ import print_function

import numpy as np
import pandas as pd

from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from keras.regularizers import l2 # L2-regularisation

from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

#import scikit-learn function
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

ImportError: cannot import name 'conditional' from 'hyperas.distributions' (/usr/local/anaconda3/lib/python3.7/site-packages/hyperas/distributions.py)

In [2]:
def create_model_RegDropout(X,y,X_val,y_val):
    model = Sequential()
    model.add(Dense(input_dim = X.shape[1], units={{choice([50, 100, 150])}}, init = "glorot_uniform")) 
    model.add(Activation({{choice(['relu','sigmoid','tanh'])}}))
    model.add(Dropout({{uniform(0.25, .75)}}))

    model.add(Dense(units={{choice([50, 100, 150])}}, init = "glorot_uniform")) 
    model.add(Activation({{choice(['relu','sigmoid','tanh'])}}))
    model.add(Dropout({{uniform(0.25, .75)}}))

    # If we choose 'four', add an additional fourth layer
    if conditional({{choice(['three', 'four'])}}) == 'four':
        model.add(Dense(units={{choice([50, 100, 150])}}, init = "glorot_uniform"))
        model.add(Activation({{choice(['relu','sigmoid','tanh'])}}))
        model.add(Dropout({{uniform(0.25, 0.75)}}))
           

    model.add(Dense(units=1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer={{choice(['rmsprop', 'adam', 'sgd'])}}, 
                  metrics=['accuracy'])

    model.fit(X, y, nb_epoch=200, verbose = 2, validation_data=(X_val, y_val))
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)

    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


def create_model_RegL2(X,y,X_val,y_val):
    model = Sequential()
    model.add(Dense(input_dim = X.shape[1], units={{choice([50, 100, 150])}}, init = "glorot_uniform",
                   kernel_regularizer=l2(l={{choice([1.e-4, 1.e-3, 1.e-2, 0.1])}}))) 
    model.add(Activation({{choice(['relu','sigmoid','tanh'])}}))

    model.add(Dense(units={{choice([50, 100, 150])}}, init = "glorot_uniform",
                   kernel_regularizer=l2(l={{choice([1.e-4, 1.e-3, 1.e-2, 0.1])}}))) 
    model.add(Activation({{choice(['relu','sigmoid','tanh'])}}))

    # If we choose 'four', add an additional fourth layer
    if conditional({{choice(['three', 'four'])}}) == 'four':
        model.add(Dense(units={{choice([50, 100, 150])}}, init = "glorot_uniform",
                       kernel_regularizer=l2(l={{choice([1.e-4, 1.e-3, 1.e-2, 0.1])}})))
        model.add(Activation({{choice(['relu','sigmoid','tanh'])}}))           

    model.add(Dense(units=1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer={{choice(['rmsprop', 'adam', 'sgd'])}}, 
                  metrics=['accuracy'])

    model.fit(X, y, nb_epoch=1, verbose = 2, validation_data=(X_val, y_val))
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)

    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [3]:
# Input data
def data():
    old = np.genfromtxt('frc_post_data.txt')
    
    # Read in the new FRC simulations
    new = np.genfromtxt('/home/sousae/frc_data/newFRCdata.txt')
    
    # merge both new and old data
    frc = np.concatenate((old,new),axis=0)
    #df = pd.DataFrame(frc, columns=['omega', 'Bomega', 'Te', 'lambda', 'gamma', 'gammaOverCrit', 'svd_bz', 
    #                            'svd_bz_per', 'svd_rho', 'svd_rho_perc', 'minBz', 'reversal'])
    
    #df["Bz_negative"] = df['reversal'].astype('category')
    
    # setup need variables 
    predict = frc[:,3:5]
    target  = to_categorical(frc[:,11])[:,1]
    
    n_cols = predict.shape[1]
    
    # scale the predictors
    scaler = StandardScaler()
    scaler.fit(predict)
    predictors_scaled = scaler.transform(predict)
    
    # do the train test split
    X, X_val, y, y_val = train_test_split(predictors_scaled,target, test_size=0.2, random_state=0)
    
    return X, y, X_val, y_val

In [4]:
# For Dropout regularization

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model_RegDropout,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=50,
                                          trials=Trials(),
                                          notebook_name='Keras_Optimization',
                                          verbose=1)
    
    X_train, Y_train, X_test, Y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, Y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    # Save model and weights
    model_json = best_model.to_json()
    with open("neuralNetModel.json", 'w') as json_file:
        json_file.write(model_json)
    # weights to HDF5
    best_model.save_weights("neuralNetModelWeights.h5")

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from keras.regularizers import l2
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform, conditional
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from matplotlib.colors import ListedColormap
except:
    pass

try:
    from keras.utils import to_categorical
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    from sklearn.preprocess

/home/sousae/projects/temp_model.py:123: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=2, units=50, kernel_initializer="glorot_uniform")`
/home/sousae/projects/temp_model.py:127: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=100, kernel_initializer="glorot_uniform")`
/home/sousae/projects/temp_model.py:133: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=150, kernel_initializer="glorot_uniform")`
/home/sousae/anaconda3/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 320 samples, validate on 80 samples
Epoch 1/200
 - 1s - loss: 0.7984 - acc: 0.5156 - val_loss: 0.6921 - val_acc: 0.5500
Epoch 2/200
 - 1s - loss: 0.7908 - acc: 0.5125 - val_loss: 0.6906 - val_acc: 0.5500
Epoch 3/200
 - 1s - loss: 0.7766 - acc: 0.5281 - val_loss: 0.6910 - val_acc: 0.5500
Epoch 4/200
 - 0s - loss: 0.7461 - acc: 0.5406 - val_loss: 0.6898 - val_acc: 0.5500
Epoch 5/200
 - 0s - loss: 0.7397 - acc: 0.5219 - val_loss: 0.6892 - val_acc: 0.5500
Epoch 6/200
 - 0s - loss: 0.7976 - acc: 0.4844 - val_loss: 0.6896 - val_acc: 0.5500
Epoch 7/200
 - 0s - loss: 0.7393 - acc: 0.5187 - val_loss: 0.6895 - val_acc: 0.5500
Epoch 8/200
 - 0s - loss: 0.7811 - acc: 0.4875 - val_loss: 0.6891 - val_acc: 0.5500
Epoch 9/200
 - 0s - loss: 0.7403 - acc: 0.5375 - val_loss: 0.6898 - val_acc: 0.5500
Epoch 10/200
 - 1s - loss: 0.7551 - acc: 0.5219 - val_loss: 0.6898 - val_acc: 0.5500
Epoch 11/200
 - 0s - loss: 0.7730 - acc: 0.4844 - val_loss: 0.6888 - val_acc: 0.5500
Epoch 12/200
 - 0s - loss: 0.

/home/sousae/projects/temp_model.py:123: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=2, units=100, kernel_initializer="glorot_uniform")`
/home/sousae/projects/temp_model.py:127: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=150, kernel_initializer="glorot_uniform")`


Train on 320 samples, validate on 80 samples
Epoch 1/200
 - 1s - loss: 0.7234 - acc: 0.5531 - val_loss: 0.5857 - val_acc: 0.8250
Epoch 2/200
 - 0s - loss: 0.6476 - acc: 0.6281 - val_loss: 0.5269 - val_acc: 0.8750
Epoch 3/200
 - 0s - loss: 0.5576 - acc: 0.7000 - val_loss: 0.4690 - val_acc: 0.8500
Epoch 4/200
 - 0s - loss: 0.5370 - acc: 0.7250 - val_loss: 0.4254 - val_acc: 0.8875
Epoch 5/200
 - 0s - loss: 0.4742 - acc: 0.7844 - val_loss: 0.3803 - val_acc: 0.8875
Epoch 6/200
 - 0s - loss: 0.4093 - acc: 0.8438 - val_loss: 0.3502 - val_acc: 0.9000
Epoch 7/200
 - 0s - loss: 0.4073 - acc: 0.8187 - val_loss: 0.3219 - val_acc: 0.8875
Epoch 8/200
 - 0s - loss: 0.3993 - acc: 0.8219 - val_loss: 0.3007 - val_acc: 0.8875
Epoch 9/200
 - 0s - loss: 0.3732 - acc: 0.8187 - val_loss: 0.2849 - val_acc: 0.8875
Epoch 10/200
 - 0s - loss: 0.3458 - acc: 0.8531 - val_loss: 0.2749 - val_acc: 0.9000
Epoch 11/200
 - 0s - loss: 0.3353 - acc: 0.8719 - val_loss: 0.2682 - val_acc: 0.8875
Epoch 12/200
 - 0s - loss: 0.

...

Train on 320 samples, validate on 80 samples
Epoch 1/200
 - 1s - loss: 0.5616 - acc: 0.7000 - val_loss: 0.3640 - val_acc: 0.8625
Epoch 2/200
 - 0s - loss: 0.3847 - acc: 0.8344 - val_loss: 0.2757 - val_acc: 0.9125
Epoch 3/200
 - 0s - loss: 0.3524 - acc: 0.8375 - val_loss: 0.2539 - val_acc: 0.8875
Epoch 4/200
 - 0s - loss: 0.3482 - acc: 0.8438 - val_loss: 0.2458 - val_acc: 0.8875
Epoch 5/200
 - 0s - loss: 0.3114 - acc: 0.8625 - val_loss: 0.2456 - val_acc: 0.8875
Epoch 6/200
 - 0s - loss: 0.3519 - acc: 0.8562 - val_loss: 0.2519 - val_acc: 0.8875
Epoch 7/200
 - 0s - loss: 0.3152 - acc: 0.8719 - val_loss: 0.2486 - val_acc: 0.8875
Epoch 8/200
 - 0s - loss: 0.3289 - acc: 0.8656 - val_loss: 0.2464 - val_acc: 0.8875
Epoch 9/200
 - 0s - loss: 0.3133 - acc: 0.8875 - val_loss: 0.2470 - val_acc: 0.8875
Epoch 10/200
 - 0s - loss: 0.3063 - acc: 0.8562 - val_loss: 0.2498 - val_acc: 0.8875
Epoch 11/200
 - 0s - loss: 0.3218 - acc: 0.8656 - val_loss: 0.2464 - val_acc: 0.8875
Epoch 12/200
 - 1s - loss: 0.

/home/sousae/projects/temp_model.py:133: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=50, kernel_initializer="glorot_uniform")`


Train on 320 samples, validate on 80 samples
Epoch 1/200
 - 1s - loss: 0.8785 - acc: 0.4750 - val_loss: 0.6761 - val_acc: 0.5500
Epoch 2/200
 - 0s - loss: 0.7790 - acc: 0.5344 - val_loss: 0.6745 - val_acc: 0.5500
Epoch 3/200
 - 0s - loss: 0.7282 - acc: 0.5344 - val_loss: 0.6714 - val_acc: 0.5500
Epoch 4/200
 - 0s - loss: 0.7548 - acc: 0.5219 - val_loss: 0.6707 - val_acc: 0.6000
Epoch 5/200
 - 0s - loss: 0.7255 - acc: 0.5312 - val_loss: 0.6706 - val_acc: 0.8000
Epoch 6/200
 - 0s - loss: 0.7313 - acc: 0.5312 - val_loss: 0.6676 - val_acc: 0.7000
Epoch 7/200
 - 0s - loss: 0.6809 - acc: 0.5844 - val_loss: 0.6656 - val_acc: 0.6625
Epoch 8/200
 - 0s - loss: 0.7253 - acc: 0.5156 - val_loss: 0.6619 - val_acc: 0.6125
Epoch 9/200
 - 0s - loss: 0.7150 - acc: 0.5625 - val_loss: 0.6607 - val_acc: 0.5875
Epoch 10/200
 - 0s - loss: 0.7115 - acc: 0.5312 - val_loss: 0.6603 - val_acc: 0.7125
Epoch 11/200
 - 0s - loss: 0.7102 - acc: 0.5438 - val_loss: 0.6582 - val_acc: 0.7125
Epoch 12/200
 - 0s - loss: 0.

/home/sousae/projects/temp_model.py:127: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=50, kernel_initializer="glorot_uniform")`


Train on 320 samples, validate on 80 samples
Epoch 1/200
 - 1s - loss: 0.8609 - acc: 0.4969 - val_loss: 0.6993 - val_acc: 0.4500
Epoch 2/200
 - 0s - loss: 0.8538 - acc: 0.4844 - val_loss: 0.6934 - val_acc: 0.4500
Epoch 3/200
 - 0s - loss: 0.7851 - acc: 0.5375 - val_loss: 0.6909 - val_acc: 0.4500
Epoch 4/200
 - 0s - loss: 0.8893 - acc: 0.4406 - val_loss: 0.6878 - val_acc: 0.5500
Epoch 5/200
 - 0s - loss: 0.8265 - acc: 0.5156 - val_loss: 0.6847 - val_acc: 0.5500
Epoch 6/200
 - 0s - loss: 0.8220 - acc: 0.4844 - val_loss: 0.6843 - val_acc: 0.5500
Epoch 7/200
 - 0s - loss: 0.7968 - acc: 0.5531 - val_loss: 0.6841 - val_acc: 0.5500
Epoch 8/200
 - 0s - loss: 0.7688 - acc: 0.5250 - val_loss: 0.6839 - val_acc: 0.5500
Epoch 9/200
 - 0s - loss: 0.7492 - acc: 0.5281 - val_loss: 0.6840 - val_acc: 0.5500
Epoch 10/200
 - 0s - loss: 0.8226 - acc: 0.5062 - val_loss: 0.6837 - val_acc: 0.5500
Epoch 11/200
 - 0s - loss: 0.7931 - acc: 0.5094 - val_loss: 0.6838 - val_acc: 0.5500
Epoch 12/200
 - 0s - loss: 0.

# For L-2 regularization
```python
if __name__ == '__main__':
    best_run2, best_model2 = optim.minimize(model=create_model_RegL2,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=50,
                                          trials=Trials(),
                                          notebook_name='Keras_Optimization',
                                          verbose=1)
    
    X_train, Y_train, X_test, Y_test = data()
    print("Evalutation of best performing model:")
    print(best_model2.evaluate(X_test, Y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run2)
```

In [5]:
# predict the classes
y_eval = best_model.predict_classes(X_test)

In [6]:
# Making the confusion Matrix
cm = confusion_matrix(Y_test, y_eval)
print(cm)

[[39  5]
 [ 2 34]]
